In [8]:
import os, sys
from pathlib import Path

def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")

src_path = find_src_folder()
sys.path.append(str(src_path))

In [9]:
from core.common import get_best_device
get_best_device()

device(type='mps')

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code = True,
    device_map=get_best_device()
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.55s/it]


In [25]:
def correct_text(model, tokenizer, text: str) -> str:
    # Exaone 4.0 Instruct에 가장 잘 먹히는 교정 프롬프트
    prompt = f"""
    당신은 STT로 추출한 내용의 발음이 이상한지 판단해서 이상하면 재교정하는 한국어 전문가 입니다.
    어색할 수 있는 다음 문장을 교정해보세요. 
    질문:{text}
    
    [출력형식]
    불필요한 붙이지 말고 질문의 답만 말씀해주세요.
    ### 교정된 문장:
    """
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=False,                  # 교정 정확도 ↑
        eos_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # "교정된 문장:" 이후만 추출
    if "교정된 문장:" in decoded:
        decoded = decoded.split("교정된 문장:")[-1].strip()
        
    return decoded 


In [30]:
correct_text(model,tokenizer,"내가 아꽈 공격 하라했자나 너 그거를 목아라 머거?")

'내가 공격하라고 했자나, 너는 그걸 받아 먹어라.'